# OpenAI function calling for Sub-Question Query Engine

In this notebook, we showcase how to use OpenAI function calling to improve the robustness of our sub-question query engine.  

The sub-question query engine is designed to accept swappable question generators that implement the `BaseQuestionGenerator` interface.  
To leverage the power of openai function calling API, we implemented a new `OpenAIQuestionGenerator` (powered by our `OpenAIPydanticProgram`)

## OpenAI Question Generator

Unlike the default `LLMQuestionGenerator` that supports generic LLMs via the completion API, `OpenAIQuestionGenerator` only works with the latest OpenAI models that supports the function calling API. 

The benefit is that these models are fine-tuned to output JSON objects, so we can worry less about output parsing issues.

In [37]:
from llama_index.question_gen.openai_generator import OpenAIQuestionGenerator

In [38]:
question_gen = OpenAIQuestionGenerator.from_defaults()

Let's test it out!

In [39]:
from llama_index.tools import ToolMetadata
from llama_index import QueryBundle

In [49]:
tools = [
ToolMetadata(
            name="march_22",
            description="Provides information about Uber quarterly financials ending March 2022",
        ),
ToolMetadata(
            name="june_22",
            description="Provides information about Uber quarterly financials ending June 2022",
        ),
ToolMetadata(
        name="sept_22",
        description="Provides information about Uber quarterly financials ending September 2022",
    ),
ToolMetadata(
            name="sept_21",
            description="Provides information about Uber quarterly financials ending September 2022",
        ),
ToolMetadata(
            name="june_21",
            description="Provides information about Uber quarterly financials ending June 2022",
        ),
ToolMetadata(
            name="march_21",
            description="Provides information about Uber quarterly financials ending March 2022",
        ),
]

In [52]:
sub_questions = question_gen.generate(
    tools=tools,
    query=QueryBundle("Compare the fastest growing sectors for Uber in the first two quarters of 2022"),
)

In [53]:
sub_questions

[SubQuestion(sub_question='What were the fastest growing sectors for Uber in March 2022?', tool_name='march_22'),
 SubQuestion(sub_question='What were the fastest growing sectors for Uber in June 2022?', tool_name='june_22')]

In [48]:
from llama_index import (
    SimpleDirectoryReader,
    LLMPredictor,
    ServiceContext,
    VectorStoreIndex,
)
from llama_index.response.pprint_utils import pprint_response

from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.query_engine import SubQuestionQueryEngine

In [9]:
lyft_docs = SimpleDirectoryReader(input_files=["../data/10k/lyft_2021.pdf"]).load_data()
uber_docs = SimpleDirectoryReader(input_files=["../data/10k/uber_2021.pdf"]).load_data()

KeyboardInterrupt: 

In [ ]:
lyft_index = VectorStoreIndex.from_documents(lyft_docs)
uber_index = VectorStoreIndex.from_documents(uber_docs)

In [ ]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)